<a href="https://colab.research.google.com/github/gastonviveros2411/labo2025v/blob/main/src/ensembles/Tarea_clase4_%20420_ArbolesAzarosos_final.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [13]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Tue Nov 04 04:40:14 AM 2025"

In [14]:
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,886360,47.4,1930531,103.2,1930531,103.2
Vcells,1746097,13.4,175434863,1338.5,219293573,1673.1


In [15]:
require("data.table")
require("rpart")

In [16]:
PARAM <- list()
PARAM$semilla_primigenia <- 129457  # ← TU SEMILLA

# Parámetros del ensemble
PARAM$feature_fraction <- 0.5
PARAM$num_trees_max <- 32  # Número de árboles

# ========================================
# HIPERPARÁMETROS A PROBAR
# Basados en Juan Manuel Molina (mejor de la clase)
# ========================================
PARAM$rpart$cp <- -1
PARAM$rpart$minsplit <- 50
PARAM$rpart$minbucket <- 25
PARAM$rpart$maxdepth <- 10

cat("Configuración:\n")
cat("  cp:", PARAM$rpart$cp, "\n")
cat("  minsplit:", PARAM$rpart$minsplit, "\n")
cat("  minbucket:", PARAM$rpart$minbucket, "\n")
cat("  maxdepth:", PARAM$rpart$maxdepth, "\n")
cat("  num_trees:", PARAM$num_trees_max, "\n")

Configuración:
  cp: -1 
  minsplit: 50 
  minbucket: 25 
  maxdepth: 10 
  num_trees: 32 


In [17]:
setwd("/content/buckets/b1/exp")
experimento <- "exp4020_corregido"
dir.create(experimento, showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento))

In [18]:
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [19]:
# IMPORTANTE: Usar TODO 202107 para train (no split)
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]
dfuture[, clase_ternaria := NA]

cat("Training:", nrow(dtrain), "\n")
cat("Future:", nrow(dfuture), "\n")

Training: 164596 
Future: 165093 


In [20]:
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))
cat("Campos:", length(campos_buenos), "\n")

Campos: 154 


In [21]:
# Preparar tabla de predicción
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob_acumulada := 0]

set.seed(PARAM$semilla_primigenia)

cat("\nEntrenando", PARAM$num_trees_max, "árboles...\n")

for (arbolito in seq(PARAM$num_trees_max)) {

  if (arbolito %% 4 == 0) cat(arbolito, " ")

  # Selección aleatoria de campos
  qty_campos_a_utilizar <- as.integer(length(campos_buenos) * PARAM$feature_fraction)
  campos_random <- sample(campos_buenos, qty_campos_a_utilizar)
  campos_random <- paste(campos_random, collapse=" + ")
  formulita <- paste0("clase_ternaria ~ ", campos_random)

  # Entrenar árbol
  modelo <- rpart(
    formulita,
    data=dtrain,
    xval=0,
    control=PARAM$rpart
  )

  # Predecir
  prediccion <- predict(modelo, dfuture, type="prob")

  # Acumular
  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]
}

cat("\n✓ Ensemble entrenado\n")


Entrenando 32 árboles...
4  8  12  16  20  24  28  32  
✓ Ensemble entrenado


In [22]:
# CLAVE: Usar umbral basado en probabilidad acumulada
# Típicamente entre 1/40 y 2/40 del número de árboles

umbral_corte <- (1 / 40) * PARAM$num_trees_max

cat("\nUmbral de corte:", umbral_corte, "\n")
cat("Equivale a probabilidad promedio de:", umbral_corte / PARAM$num_trees_max, "\n")

tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

cat("\nClientes predichos como BAJA+2:", sum(tb_prediccion$Predicted), "\n")


Umbral de corte: 0.8 
Equivale a probabilidad promedio de: 0.025 

Clientes predichos como BAJA+2: 10507 


In [23]:
archivo_kaggle <- paste0(
  "KA420_",
  sprintf("%03d", PARAM$num_trees_max),
  "_correcto.csv"
)

fwrite(
  tb_prediccion[, list(numero_de_cliente, Predicted)],
  file=archivo_kaggle,
  sep=","
)

cat("\nArchivo generado:", archivo_kaggle, "\n")


Archivo generado: KA420_032_correcto.csv 


In [26]:
# Subir a Kaggle
    # subida a Kaggle
    comando <- "kaggle competitions submit"
    competencia <- "-c   labo-i-2025-virtual-analista-sr"
    arch <- paste( "-f", archivo_kaggle)


mensaje <- paste0(
  "-m 'cp=", PARAM$rpart$cp,
  " minsplit=", PARAM$rpart$minsplit,
  " minbucket=", PARAM$rpart$minbucket,
  " maxdepth=", PARAM$rpart$maxdepth, "'"
)

linea <- paste(comando, competencia, arch, mensaje)
cat("\nEjecutando:", linea, "\n\n")
salida <- system(linea, intern=TRUE)
cat(salida)


Ejecutando: kaggle competitions submit -c   labo-i-2025-virtual-analista-sr -f KA420_032_correcto.csv -m 'cp=-1 minsplit=50 minbucket=25 maxdepth=10' 

Successfully submitted to LaboI 2025 virtual analista sr

In [27]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Tue Nov 04 05:24:28 AM 2025"